# ID2214/FID3214 Assignment 3 Group no. 12
### Project members: 
Aksel Uhr, auhr@kth.se
Olivia Höft, hoft@kth.se
Ilias Merentitis, iliasme@kth.se

### Declaration
By submitting this solution, it is hereby declared that all individuals listed above have contributed to the solution, either with code that appear in the final solution below, or with code that has been evaluated and compared to the final solution, but for some reason has been excluded. It is also declared that all project members fully understand all parts of the final solution and can explain it upon request.

It is furthermore declared that the code below is a contribution by the project members only, and specifically that no part of the solution has been copied from any other source (except for lecture slides at the course ID2214/FID3214) and no part of the solution has been provided by someone not listed as project member above.

It is furthermore declared that it has been understood that no other library/package than the Python 3 standard library, NumPy, pandas, time and sklearn.tree, may be used in the solution for this assignment.

### Instructions
All parts of the assignment starting with number 1 below are mandatory. Satisfactory solutions
will give 1 point (in total). If they in addition are good (all parts work more or less 
as they should), completed on time (submitted before the deadline in Canvas) and according
to the instructions, together with satisfactory solutions of all parts of the assignment starting 
with number 2 below, then the assignment will receive 2 points (in total).

Note that you do not have to develop the code directly within the notebook
but may instead copy the comments and test cases to a more convenient development environment
and when everything works as expected, you may paste your functions into this
notebook, do a final testing (all cells should succeed) and submit the whole notebook 
(a single file) in Canvas (do not forget to fill in your group number and names above).

## Load NumPy, pandas, time and DecisionTreeClassifier from sklearn.tree

In [ ]:
import numpy as np
import pandas as pd
import time
from sklearn.tree import DecisionTreeClassifier
import math

## Reused functions from Assignment 1

In [ ]:
# Copy and paste functions from Assignment 1 here that you need for this assignment
def create_column_filter(df):
    
    # Copies the input df
    dfCopy = df.copy()  
   
    # Drops each column with only one unique value + one unique value with nan(s)
    for col in dfCopy.columns:
        if (col != "CLASS" and col != "ID") and (len(dfCopy[col].unique()) == 1 or (len(dfCopy[col].unique()) == 2 and dfCopy.isnull().values.any())):
            dfCopy.drop(col,inplace=True,axis=1)
             
    # Extracting column names and converts to list
    remainingColumns = dfCopy.columns.values.tolist()
    return dfCopy, remainingColumns

def apply_column_filter(df, column_filter):
    
    # Copies the input df
    dfCopy = df.copy()
    
    # Finds all similar column names and keep the values of them
    # Discards the rest
    df_final = dfCopy[dfCopy.columns.intersection(column_filter)]
    return df_final

def create_imputation(df):
    dfCopy = df.copy()
    dictionary = {}
    
    for col in dfCopy:
        
        # Checks type and column name
        if (col != "CLASS" and col != "ID") and (dfCopy[col].dtype == np.int64 or dfCopy[col].dtype == np.float64):
            if dfCopy[col].isnull().all():
                dfCopy[col].fillna(0, inplace=True)
                dictionary[col] = 0 
            else:
                mean = dfCopy[col].mean()
                dfCopy[col].fillna(mean, inplace=True)
                dictionary[col] = mean 
        elif (col != "CLASS" and col != "ID") and (dfCopy[col].dtype == np.object):
            if dfCopy[col].isnull().all():
                dfCopy[col].fillna('', inplace=True)
                dictionary[col] = '' 
            else:
                mode = dfCopy[col].mode()
                dfCopy[col].fillna(mode[0], inplace=True)
                dictionary[col] = mode[0] 
                    
    return dfCopy, dictionary

def apply_imputation(df, dictionary):
    dfCopy = df.copy()
    
    for col in dictionary:
        dfCopy[col].fillna(dictionary[col], inplace=True)
        
    return dfCopy

from pandas.api.types import is_numeric_dtype

def create_one_hot(df):
    dfCopy = df.copy()
    one_hot = {}
    
    for col in dfCopy.columns:
        if (col != "CLASS" and col != "ID") and not is_numeric_dtype(dfCopy[col]):

            dfCopy[col] = dfCopy[col].astype("category")
            one_hot[col]= list(dfCopy[col].cat.categories)

            y = pd.get_dummies(dfCopy[col], prefix = col, dtype=float)
            dfCopy = pd.concat([dfCopy, y], axis=1)
            
            dfCopy.drop(col, inplace = True, axis = 1)
            
    return dfCopy, one_hot

def apply_one_hot(df, one_hot):
    dfCopy = df.copy()
    
    for col in one_hot:
        y = pd.get_dummies(dfCopy[col], dtype=float)
        for cat in one_hot[col]:
            if cat in y:
                dfCopy[col+'-'+str(cat)] = y[cat]
            else:
                dfCopy[col+'-'+str(cat)] = 0
    
        dfCopy.drop(col, inplace = True, axis = 1)

    return dfCopy

def accuracy(df, correctlabels):
    dfCopy = df.copy()
    
    highestLabels = dfCopy.idxmax(axis=1)
    highestLabelsList = highestLabels.values.tolist()
    
    count = 0
    dfLength = len(dfCopy)
    
    for i in range(len(correctlabels)):
        if correctlabels[i] == highestLabelsList[i]:
            count = count + 1
    
    res = count / dfLength
    
    return res

def brier_score(df, correctlabels):
    dfCopy = df.copy()
    
    correctDf = pd.get_dummies(correctlabels)
    
    brier_score = np.mean(np.sum((dfCopy-correctDf)**2, axis= 1))
    
    return brier_score

def get_pos_and_neg(predictions, pos_label, correct_labels):
    scores = {}
    
    # Counts tps amd fps and put them in a dict
    for i in range(len(predictions)):
        score = predictions[pos_label][i] 
        (p,n) = scores.get(score,(0,0))
        if correct_labels[i] == pos_label:
            scores[score] = (p+1,n)
        else:
            scores[score] = (p,n+1)

    # dict to array
    # sort array
    scores = np.array([[score,scores[score][0],scores[score][1]] for score in scores.keys()])
    scores = scores[np.argsort(scores[:,0])[::-1]]
    
    # drop predictions
    pos = scores[:,1]
    neg = scores[:,2]
    
    return pos, neg


def auc(predictions, correctlabels):
    
    AUC = 0
    
    for col in predictions.columns:
        
        #Räknar ut antal labels i correct labels
        class_frequency = correctlabels.value_counts()[col]
        
        # Gets total true positives and true negatives for each column
        pos, neg = get_pos_and_neg(predictions,col, correctlabels)
        
        # Area for each column
        thisAUC = calculateAUC(neg, pos, 0)
        
        # Total area for the predictions
        # "How many labels (e.g. A) do we have in correct labels (in 
        # This step considers ALL classes and calculates tot AUC, considering each column
        AUC += thisAUC*(class_frequency/len(correctlabels))

    return AUC
    
    
def calculateAUC(neg, pos, AUC):
    Tot_tp = sum(pos)
    Tot_fp = sum(neg)
    
    Cov_tp = 0
    for i in range(len(pos)):
        
        # Cov tp holds true positives
        AUC += (Cov_tp/Tot_tp)*(neg[i]/Tot_fp)+ (pos[i]/Tot_tp)*(neg[i]/Tot_fp)/2
        Cov_tp += pos[i]

    return AUC

## 1. Define the class RandomForest

In [ ]:
# Define the class RandomForest with three functions __init__, fit and predict (after the comments):
#
# Input to __init__: 
# self - the object itself
#
# Output from __init__:
# <nothing>
# 
# This function does not return anything but just initializes the following attributes of the object (self) to None:
# column_filter, imputation, one_hot, labels, model
#
# Input to fit:
# self      - the object itself
# df        - a dataframe (where the column names "CLASS" and "ID" have special meaning)
# no_trees  - no. of trees in the random forest (default = 100)
#
# Output from fit:
# <nothing>
#
# The result of applying this function should be:
#
# self.column_filter - a column filter (see Assignment 1) from df
# self.imputation    - an imputation mapping (see Assignment 1) from df
# self.one_hot       - a one-hot mapping (see Assignment 1) from df
# self.labels        - a (sorted) list of the categories of the "CLASS" column of df
# self.model         - a random forest, consisting of no_trees trees, where each tree is generated from a bootstrap sample
#                      and the number of evaluated features is log2|F| where |F| is the total number of features
#                      (for details, see lecture slides)
#
# Note that the function does not return anything but just assigns values to the attributes of the object.
#
# Hint 1: First create the column filter, imputation and one-hot mappings
#
# Hint 2: Then get the class labels and the numerical values (as an ndarray) from the dataframe after dropping the class labels 
#
# Hint 3: Generate no_trees classification trees, where each tree is generated using DecisionTreeClassifier 
#         from a bootstrap sample (see lecture slides), e.g., generated by np.random.choice (with replacement) 
#         from the row numbers of the ndarray, and where a random sample of the features are evaluated in
#         each node of each tree, of size log2(|F|), where |F| is the total number of features;
#         see the parameter max_features of DecisionTreeClassifier
#
# Input to predict:
# self - the object itself
# df   - a dataframe
# 
# Output from predict:
# predictions - a dataframe with class labels as column names and the rows corresponding to
#               predictions with estimated class probabilities for each row in df, where the class probabilities
#               are the averaged probabilities output by each decision tree in the forest
#
# Hint 1: Drop any "CLASS" and "ID" columns of the dataframe first and then apply column filter, imputation and one_hot
#
# Hint 2: Iterate over the trees in the forest to get the prediction of each tree by the method predict_proba(X) where 
#         X are the (numerical) values of the transformed dataframe; you may get the average predictions of all trees,
#         by first creating a zero-matrix with one row for each test instance and one column for each class label, 
#         to which you add the prediction of each tree on each iteration, and then finally divide the prediction matrix
#         by the number of trees.
#
# Hint 3: You may assume that each bootstrap sample that was used to generate each tree has included all possible
#         class labels and hence the prediction of each tree will contain probabilities for all class labels
#         (in the same order). Note that this assumption may be violated, and this limitation will be addressed 
#         in the next part of the assignment.

class RandomForest:
    
    def __init__(self):
        self.column_filter = None
        self.imputation = None
        self.one_hot = None
        self.labels = None
        self.model = None
        
    def fit(self, df, no_trees = 100):
        df, self.column_filter = create_column_filter(df)
        df, self.imputation = create_imputation(df)
        df, self.one_hot = create_one_hot(df)
        
        self.labels = np.sort(df.CLASS).tolist()
        
        dfCopy = df.drop(columns = ["CLASS"])
        
        numCol = len(df.columns)
        
        
        self.model = list()
        for i in range(no_trees):
            sample = df.sample(n = len(df), replace = True)
            y = sample["CLASS"]
            x = sample.drop(columns = ["CLASS"])
            clf = DecisionTreeClassifier(max_features = math.floor(math.log2(numCol)))
            clf = clf.fit(x,y)
            self.model.append(clf)
            
            
    def predict(self, df):
        dfCopy = df.copy()
        
        dfCopy = dfCopy.drop(columns = ["CLASS"])
        
        dfCopy = apply_column_filter(dfCopy, self.column_filter)
        dfCopy = apply_imputation(dfCopy, self.imputation)
        dfCopy = apply_one_hot(dfCopy, self.one_hot)
        
        npPredictions = np.zeros((len(dfCopy), len(np.unique(self.labels))))
        
        prediction = list()
   
        for i in range (len(self.model)):
            prediction = self.model[i].predict_proba(dfCopy)
            npPredictions += prediction
        
        npPredictions /= len(self.model)
        dfResult = pd.DataFrame(npPredictions, columns = np.unique(self.labels))
        
        return dfResult



In [ ]:
# Test your code (leave this part unchanged, except for if auc is undefined)

train_df = pd.read_csv("tic-tac-toe_train.csv")

test_df = pd.read_csv("tic-tac-toe_test.csv")

rf = RandomForest()

t0 = time.perf_counter()
rf.fit(train_df)
print("Training time: {:.2f} s.".format(time.perf_counter()-t0))

test_labels = test_df["CLASS"]

t0 = time.perf_counter()
predictions = rf.predict(test_df)

print("Testing time: {:.2f} s.".format(time.perf_counter()-t0))

print("Accuracy: {:.4f}".format(accuracy(predictions,test_labels)))
print("AUC: {:.4f}".format(auc(predictions,test_labels))) # Comment this out if not implemented in assignment 1
print("Brier score: {:.4f}".format(brier_score(predictions,test_labels))) # Comment this out if not implemented in assignment 1

In [ ]:
train_labels = train_df["CLASS"]
predictions = rf.predict(train_df)
print("Accuracy on training set: {0:.4f}".format(accuracy(predictions,train_labels)))
print("AUC on training set: {0:.4f}".format(auc(predictions,train_labels))) # Comment this out if not implemented in assignment 1
print("Brier score on training set: {0:.4f}".format(brier_score(predictions,train_labels))) # Comment this out if not implemented in assignment 1

### Comment on assumptions, things that do not work properly, etc.


## 2a. Handling trees with non-aligned predictions

In [ ]:
# Define a revised version of the class RandomForest with the same input and output as described in part 1 above,
# where the predict function is able to handle the case where the individual trees are trained on bootstrap samples
# that do not include all class labels in the original training set. This leads to that the class probabilities output
# by the individual trees in the forest do not refer to the same set of class labels.
#
# Hint 1: The categories obtained with <pandas series>.cat.categories are sorted in the same way as the class labels
#         of a DecisionTreeClassifier; the latter are obtained by <DecisionTreeClassifier>.classes_ 
#         The problem is that classes_ may not include all possible labels, and hence the individual predictions 
#         obtained by <DecisionTreeClassifier>.predict_proba may be of different length or even if they are of the same
#         length do not necessarily refer to the same class labels. You may assume that each class label that is not included
#         in a bootstrap sample should be assigned zero probability by the tree generated from the bootstrap sample. 
#
# Hint 2: Create a mapping from the complete (and sorted) set of class labels l0, ..., lk-1 to a set of indexes 0, ..., k-1,
#         where k is the number of classes
#
# Hint 3: For each tree t in the forest, create a (zero) matrix with one row per test instance and one column per class label,
#         to which one column is added at a time from the output of t.predict_proba 
#
# Hint 4: For each column output by t.predict_proba, its index i may be used to obtain its label by t.classes_[i];
#         you may then obtain the index of this label in the ordered list of all possible labels from the above mapping (hint 2); 
#         this index points to which column in the prediction matrix the output column should be added to 

class RandomForest:
    
    def __init__(self):
        self.column_filter = None
        self.imputation = None
        self.one_hot = None
        self.labels = None
        self.model = None
        
    def fit(self, df, no_trees = 100):
        df, self.column_filter = create_column_filter(df)
        df, self.imputation = create_imputation(df)
        df, self.one_hot = create_one_hot(df)
        
        self.labels = np.sort(np.unique(df.CLASS)).tolist()
        
        dfCopy = df.drop(columns = ["CLASS"])
        
        numCol = len(df.columns)
        
        self.model = list()
        for i in range(no_trees):
            sample = df.sample(n = len(df), replace = True)
            y = sample["CLASS"]
            x = sample.drop(columns = ["CLASS"])
            clf = DecisionTreeClassifier(max_features = math.floor(math.log2(numCol)))
            clf = clf.fit(x,y)
            self.model.append(clf)
            
            
    def predict(self, df):
        dfCopy = df.copy()

        dfCopy = dfCopy.drop(columns = ["CLASS"])

        dfCopy = apply_column_filter(dfCopy, self.column_filter)
        dfCopy = apply_imputation(dfCopy, self.imputation)
        dfCopy = apply_one_hot(dfCopy, self.one_hot)

        npPredictions = np.zeros((len(dfCopy), len(np.unique(self.labels))))

        prediction = list()

        for i in range (len(self.model)):
            prediction = self.model[i].predict_proba(dfCopy)

            tmp = list()
            for j in range (len(prediction[0])):
                tmp.append(self.labels.index(self.model[i].classes_[j]))
            npPredictions[:,tmp] += prediction
                
                      
        npPredictions /= len(self.model)
        dfResult = pd.DataFrame(npPredictions, columns = np.unique(self.labels))
        
        return dfResult

In [ ]:
# Test your code (leave this part unchanged, except for if auc is undefined)

train_df = pd.read_csv("anneal_train.csv")

test_df = pd.read_csv("anneal_test.csv")

rf = RandomForest()

t0 = time.perf_counter()
rf.fit(train_df)
print("Training time: {:.2f} s.".format(time.perf_counter()-t0))

test_labels = test_df["CLASS"]

t0 = time.perf_counter()
predictions = rf.predict(test_df)
print("Testing time: {:.2f} s.".format(time.perf_counter()-t0))

print("Accuracy: {:.4f}".format(accuracy(predictions,test_labels)))
print("AUC: {:.4f}".format(auc(predictions,test_labels))) # Comment this out if not implemented in assignment 1
print("Brier score: {:.4f}".format(brier_score(predictions,test_labels))) # Comment this out if not implemented in assignment 1

## 2b. Estimate predictive performance using out-of-bag predictions

In [ ]:
# Define an extended version of the class RandomForest with the same input and output as described in part 2a above,
# where the results of the fit function also should include:
# self.oob_acc - the accuracy estimated on the out-of-bag predictions, i.e., the fraction of training instances for 
#                which the given (correct) label is the same as the predicted label when using only trees for which
#                the instance is out-of-bag
#
# Hint 1: You may first create a zero matrix with one row for each training instance and one column for each class label
#         and one zero vector to allow for storing aggregated out-of-bag predictions and the number of out-of-bag predictions
#         for each training instance, respectively
#
# Hint 2: After generating a tree in the forest, iterate over the indexes that were not included in the bootstrap sample
#         and add a prediction of the tree to the out-of-bag prediction matrix and update the count vector
#
# Hint 3: Note that the input to predict_proba has to be a matrix; from a single vector (row) x, a matrix with one row
#         can be obtained by x[None,:]
#
# Hint 4: Finally, divide each row in the out-of-bag prediction matrix with the corresponding element of the count vector

class RandomForest:
    
    def __init__(self):
        self.column_filter = None
        self.imputation = None
        self.one_hot = None
        self.labels = None
        self.model = None
        self.oob_acc = None
        
    def fit(self, df, no_trees = 100):
        df, self.column_filter = create_column_filter(df)
        df, self.imputation = create_imputation(df)
        df, self.one_hot = create_one_hot(df)
        
        self.labels = np.sort(np.unique(df.CLASS)).tolist()
        correctLabels = df["CLASS"]
        dfCopy = df.drop(columns = ["CLASS"])        
        numCol = len(df.columns)
        
        #zero matrix    
        npPredictions = np.zeros((len(df), len(self.labels)))
        count = np.zeros(len(df))
        
        self.model = list()
        for i in range(no_trees):
            sample = df.sample(n = len(df), replace = True)
            temp = np.unique(sample.index)

            dfOutofBag = dfCopy.loc[~dfCopy.index.isin(temp)]
            
            y = sample["CLASS"]
            x = sample.drop(columns = ["CLASS"])
            clf = DecisionTreeClassifier(max_features = math.floor(math.log2(numCol)))
            clf = clf.fit(x,y)
            self.model.append(clf)
            prediction = clf.predict_proba(dfOutofBag)
            
            rows = list()
            for i, row in dfOutofBag.iterrows():
                rows.append(i)
            
            for z in range (len(dfOutofBag)):
                tmp = list()
                for j in range (len(prediction[0])):
                    tmp.append(self.labels.index(clf.classes_[j]))
                npPredictions[rows[z], tmp] += prediction[z]
            
            count[dfOutofBag.index] += 1
            
        npPredictions = npPredictions / count[:, None]
        dfResult = pd.DataFrame(npPredictions, columns = np.unique(self.labels))
        
        indexes = np.unique(dfResult.index)
        correct = correctLabels.loc[correctLabels.index.isin(indexes)]
        self.oob_acc = accuracy(dfResult, correct)
            
             
    def predict(self, df):
        dfCopy = df.copy()

        dfCopy = dfCopy.drop(columns = ["CLASS"])

        dfCopy = apply_column_filter(dfCopy, self.column_filter)
        dfCopy = apply_imputation(dfCopy, self.imputation)
        dfCopy = apply_one_hot(dfCopy, self.one_hot)

        npPredictions = np.zeros((len(dfCopy), len(self.labels)))

        prediction = list()

        for i in range (len(self.model)):
            prediction = self.model[i].predict_proba(dfCopy)
            
            

            tmp = list()
            for j in range (len(prediction[0])):
                tmp.append(self.labels.index(self.model[i].classes_[j]))
            npPredictions[:,tmp] += prediction
                
                      
        npPredictions /= len(self.model)
        dfResult = pd.DataFrame(npPredictions, columns = np.unique(self.labels))
        
        return dfResult

In [ ]:
# Test your code (leave this part unchanged, except for if auc is undefined)

train_df = pd.read_csv("anneal_train.csv")

test_df = pd.read_csv("anneal_test.csv")

rf = RandomForest()

t0 = time.perf_counter()
rf.fit(train_df)
print("Training time: {:.2f} s.".format(time.perf_counter()-t0))

print("OOB accuracy: {:.4f}".format(rf.oob_acc))

test_labels = test_df["CLASS"]

t0 = time.perf_counter()
predictions = rf.predict(test_df)
print("Testing time: {:.2f} s.".format(time.perf_counter()-t0))

print("Accuracy: {:.4f}".format(accuracy(predictions,test_labels)))
print("AUC: {:.4f}".format(auc(predictions,test_labels))) # Comment this out if not implemented in assignment 1
print("Brier score: {:.4f}".format(brier_score(predictions,test_labels))) # Comment this out if not implemented in assignment 1

In [ ]:
train_labels = train_df["CLASS"]
rf = RandomForest()
rf.fit(train_df)
predictions = rf.predict(train_df)
print("Accuracy on training set: {0:.2f}".format(accuracy(predictions,train_labels)))
print("AUC on training set: {0:.2f}".format(auc(predictions,train_labels)))
print("Brier score on training set: {0:.2f}".format(brier_score(predictions,train_labels)))

### Comment on assumptions, things that do not work properly, etc.